# Accuracy and robustness
The VMM should have a high accuracy in describing the known data, being the manoeuvring model tests in this case. A prediction model is however of little use if it can only predict data that is already known. Therefore also the robustness, how the prediction model is performing in new situations must also be considered. The accuracy of how well the VMM:s describe known data will be first analyzed. After that the robustness of the models will also be considered. 

In [1]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)

import pandas as pd
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)

from myst_nb import glue
from src.symbols import *
import src.symbols as symbols
from src.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from src.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]
only_joined = global_variables[
    "only_joined"
]  # (regress/predict with only models from joined runs)S

2022-02-08 13:12:18,790 - kedro.framework.session.store - INFO - `read()` not implemented for `SQLiteStore`. Assuming empty store.
2022-02-08 13:12:21,362 - root - INFO - ** Kedro project wPCC_pipeline
2022-02-08 13:12:21,363 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-02-08 13:12:21,373 - root - INFO - Registered line magic `run_viz`


## Accuracy
The accuracy of the models can be measured in many ways. For standard manoeuvres {cite}`imo_standards_2002` such as zigzag-tests and turning circle tests key characteristics such as overshoot angles and tactical diameters can be used {cite}`ittc_manoeuvring_2014`. When reducing the time series into a few key characterisics a lot of information is lost. Two models with the same overshoot angles can for instance have very different trajectories. This approach is good wheen the key characteristics is the main focus, for instance when assessing the compliance with the IMO standards {cite}`imo_standards_2002`.

For other cases where the predicted trajectory needs to be very accurate or for none standard manoeuvres, the accuracy needs to be calculated from the time series in a general way. One way is to calculate position error for each DOF, as the difference between predicted position and true position. It is of course impossible to know the true position from experimental data exactly. The true position therefore assumed to be approximately the same as the Kalman smoother estimate.

$$
\vec{\epsilon} = \vec{x} - \hat{\vec{x}}
$$

$$
\vec{x} = \left[x_0, y_0, \psi \right]^T
$$

where $x$ is the true position and $\hat{x}$ is the predicted position.

This way to define accuracy is very straight forward, but has the drawback that the prediction will deviate more and more over time as the prediction error is accumulated. The prediction will eventually diverge into a complete other state where it is no longer relevant to assess the accuracy. Consider the scenario where the VMM is very inaccurate in the begining of a time series, but would in fact be very accurate if only the later part of the time series is considered. This would never be seen in the accuracy as the solution has already divereged at that point.
It is therefore better to calculate the error as the error of the acceleration calculated from the true state for each time step in the time series, so that the diverge problem is avoided:

$$
\vec{\epsilon} = \vec{\ddot{x}} - \hat{\vec{\ddot{x}}}
$$

$$
\vec{\ddot{x}} = \left[\dot{u}, \dot{v}, \dot{r} \right]^T
$$
